In [2]:
import os
import pickle
from sparta.Auxil.PeriodicityDetector import PeriodicityDetector
from sparta.UNICOR.Spectrum import Spectrum
from sparta.UNICOR.Template import Template
from sparta.Auxil.TimeSeries import TimeSeries
from sparta.Observations import Observations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import imageio

In [19]:
# DATA PARAMETERS

survey = "CORALIE" # survey name
sample_rate = 1 # sample rate of the data
min_wv = 5800 # minimum wavelength of the data
max_wv = 6000 # maximum wavelength of the data
spec_dir = r"C:\Users\kentb\Desktop\PDM\data\Coralie14\spectra\BGCru" # directory of the spectrum
obs_data = Observations(survey=survey, sample_rate=sample_rate, min_wv=min_wv, max_wv=max_wv,
                        target_visits_lib=spec_dir)
period_truth = 3.342540

let's goooo
Using target_visits_lib: C:\Users\kentb\Desktop\PDM\data\Coralie14\spectra\BGCru
Selected directory: C:\Users\kentb\Desktop\PDM\data\Coralie14\spectra\BGCru


In [20]:
# Preprocess the spectra
for i in obs_data.time_series.vals:
    i = i.InterpolateSpectrum()
    i = i.FilterSpectrum(lowcut=3, highcut=0.15, order = 1)
    i = i.ApplyCosineBell(alpha=0.3)


In [4]:
for i in obs_data.time_series.vals:
    i = i.InterpolateSpectrum()

In [21]:
#fold the times and get the new order of the indices

times_folded = [t%period_truth for t in obs_data.time_series.times]
print(obs_data.time_series.times)
print(times_folded)
print(obs_data.time_series.vals)

indices = np.argsort(times_folded).tolist()
obs_data.time_series.times = [obs_data.time_series.times[i] for i in indices]
obs_data.time_series.vals = [obs_data.time_series.vals[i] for i in indices]
print([t%period_truth for t in obs_data.time_series.times])
print(obs_data.time_series.vals)
print(len(obs_data.time_series.vals), len(obs_data.time_series.times),len(times_folded))

[0.6325060902163386, 0.7231299700215459, 1.6443333001807332, 1.724320650100708, 2.6317500597797334, 2.751030730083585, 4.655399119947106, 4.752712309826165, 5.6564583401195705, 5.747590980026871, 6.616429320070893, 6.747144460212439, 8.618693099822849, 8.713424929883331, 10.62209860002622, 10.728485059924424, 11.631612140219659, 11.680812359787524, 322.801027929876, 721.5101461801678, 722.7524692001753, 724.6315697198734, 1033.8336680498905, 1034.8468316500075, 1035.8329561399296, 1037.916550709866, 1038.8256554300897, 1039.7973202001303, 1072.7254727701657, 1073.5805994598195, 1074.7814468401484, 1075.7581932097673, 1076.6505396398716, 1077.5914504099637, 1078.7684960900806, 1079.7486988902092, 1107.6945935101248, 1108.6606577900238, 1109.6603429699317, 1110.586718949955, 1111.64593764022, 1112.6316041299142, 1116.6073385900818, 1117.5137901999988, 1118.6131395502016, 1119.534994080197, 1120.6805782001466, 2207.547720800154, 2209.65902535012, 2210.7632856499404, 2410.850162469782, 241

In [22]:
# Initialize plot
fig, ax = plt.subplots()
print(len(obs_data.time_series.vals[0].wv[0]), len(obs_data.time_series.vals[0].sp[0]),len(obs_data.time_series.vals))
wavelengths = obs_data.time_series.vals[0].wv[0]
line, = ax.plot(wavelengths, obs_data.time_series.vals[0].sp[0], lw=1, color='k')

# Animation function
def update(frame):
    line.set_xdata(obs_data.time_series.vals[frame].wv[0])
    line.set_ydata(obs_data.time_series.vals[frame].sp[0])
    max_y = np.max(obs_data.time_series.vals[frame].sp[0])
    min_y = np.min(obs_data.time_series.vals[frame].sp[0])
    min_x = 5887
    max_x = 5897
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)
    ax.set_xlabel(r'Wavelength $\AA$')
    ax.set_ylabel('Intensity')
    ax.set_title(r'Na $I-II$, BG Cru',loc='left',fontsize=8)
    ax.set_title('Time: ' + str([t%period_truth for t in obs_data.time_series.times][frame]) + ' days',loc='right',fontsize=6)
    return line,

# Set animation properties
ani = FuncAnimation(fig, update, frames=len(obs_data.time_series.vals),
                    interval=50, blit=True)

# Enable interactive mode for the notebook
%matplotlib notebook

# Display the animation
HTML(ani.to_jshtml())

ani.save('BGCru_NaII.gif', writer='pillow', fps=8)

<IPython.core.display.Javascript object>

19996 19996 403


In [131]:
plt.figure()
plt.plot(obs_data.time_series.vals[0].wv[0], obs_data.time_series.vals[0].sp[0], lw=1, color='k')
plt.show()

<IPython.core.display.Javascript object>